In [ ]:
import pandas as pd
import matplotlib.colors as mc  # For the legend
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import yaml
import pypsa
import calendar

from matplotlib.cm import ScalarMappable
from pypsa.descriptors import Dict

### Select run & Prepare data

In [ ]:
def load_configuration(config_path):
    """
    Load configuration settings from a YAML file.
    """
    with open(config_path, "r") as f:
        config = yaml.safe_load(f)
    return config


snakemake_config = load_configuration("../config.yaml")
snakemake.input = Dict()
snakemake.output = Dict()

In [ ]:
run = "test-IEDK-1H-cfe100-allflex"  # run name from config.yaml

if True:
    folder = f"/results/{run}"
    scenario = "/2025/IEDK/p1/cfe100"

    snakemake.input.data = f"{folder}/networks/{scenario}/40.nc"
    snakemake.output.plot = f"{folder}/plots/plot.pdf"

    n = pypsa.Network(f"../{folder}/networks/{scenario}/40.nc")

### Explore the data

In [ ]:
n.generators_t.keys()

In [ ]:
# display two dataframes containing the time series
df = n.generators_t.p_max_pu.filter(regex="Ireland|Denmark")
df

In [ ]:
df = (
    n.generators_t.p_max_pu.filter(regex="Ireland|Denmark")
    .reset_index()
    .rename(columns={"index": "snapshot"})
    .assign(snapshot=lambda x: pd.to_datetime(x["snapshot"]))
)
df

### Visualise time-series of RES feed-in

In [ ]:
def prepare_heatmap_data(df, month, location, carrier, scaling):
    data = df[df["snapshot"].dt.month == month]
    day = data["snapshot"].dt.day
    value = data[f"{location} {carrier}"].values.reshape(
        int(24 / scaling), len(day.unique()), order="F"
    )
    return day, value


def draw_heatmap(ax, day, value, scaling, colormap, min_val, max_val):
    xgrid = np.arange(day.max() + 1) + 1  # for days
    ygrid = np.arange(int(24 / scaling) + 1)  # for hours

    # Ensure the dimensions of 'value' match the expected dimensions for 'xgrid' and 'ygrid'
    if value.shape != (len(ygrid) - 1, len(xgrid) - 1):
        raise ValueError(
            f"Shape of value ({value.shape}) does not match xgrid ({len(xgrid)}) and ygrid ({len(ygrid)}) dimensions."
        )

    ax.pcolormesh(xgrid, ygrid, value, cmap=colormap, vmin=min_val, vmax=max_val)
    ax.set_ylim(int(24 / scaling), 0)
    ax.axis("off")


def plot_heatmap_cf(
    df,
    location,
    carrier,
    scaling,
    colormap,
    min_val,
    max_val,
    year=2013,
    figsize=(14, 5),
):
    fig, axes = plt.subplots(1, 12, figsize=figsize, sharey=True)
    plt.tight_layout()

    for month, ax in enumerate(axes, start=1):
        day, value = prepare_heatmap_data(df, month, location, carrier, scaling)
        draw_heatmap(ax, day, value, scaling, colormap, min_val, max_val)
        ax.set_title(calendar.month_abbr[month], fontsize=10, pad=3)

    fig.subplots_adjust(
        left=0.05, right=0.98, top=0.9, hspace=0.08, wspace=0.1, bottom=0.15
    )
    cbar_ax = fig.add_axes([0.3, 0.08, 0.4, 0.04])
    norm = mc.Normalize(min_val, max_val)
    cb = fig.colorbar(
        ScalarMappable(norm=norm, cmap=colormap), cax=cbar_ax, orientation="horizontal"
    )
    cb.set_label("Hourly Capacity Factor (%)", size=14)

    fig.text(0.12, 0.12, "Day of the Month", ha="center", va="center", fontsize=14)
    fig.text(
        0.04,
        0.34,
        "Hour of the Day",
        ha="center",
        va="center",
        rotation="vertical",
        fontsize=14,
    )

    annotations = [
        f"Location: {location}",
        f"Carrier: {carrier}",
        f"Weather Year: {year}",
        r"Unit: MWh·h$^{-1}$",
    ]
    for i, annotation in enumerate(annotations):
        fig.text(
            0.95,
            0.12 - i * 0.05,
            annotation,
            ha="right",
            va="center",
            fontsize=14,
            color="black",
        )

    fig.savefig("test.pdf", bbox_inches="tight", transparent=True)

In [ ]:
location = "Denmark"
carrier = "solar"
scaling = int(snakemake.config["time_sampling"][0])  # temporal scaling -- 3/1 for 3H/1H
colormap = "cividis"  # https://matplotlib.org/stable/tutorials/colors/colormaps.html
MIN, MAX = 0.0, 1.0  #  df["denmark onwind"].min()

plot_heatmap_cf(df, location, carrier, scaling, colormap, MIN, MAX)

### Difference in hourly RES potentials

In [ ]:
data = df[(df["snapshot"].dt.month == 1)]
value = data[f"Denmark {carrier}"] - data[f"Ireland {carrier}"]
value.describe()

In [ ]:
MIN = -1.0  # df["denmark onwind"].min() #case of cf -> 0
MAX = 1.0  # df["denmark onwind"].max() #case of cf -> 1
colormap = "RdBu"

In [ ]:
def heatmap_diff(data, month, year, ax):
    data = df[(df["snapshot"].dt.month == month)]

    snapshot = data["snapshot"]
    day = data["snapshot"].dt.day
    value = data[f"Denmark {carrier}"] - data[f"Ireland {carrier}"]
    value = value.values.reshape(
        int(24 / scaling), len(day.unique()), order="F"
    )  # 8 clusters of 3h in each day

    xgrid = (
        np.arange(day.max() + 1) + 1
    )  # The inner + 1 increases the length, the outer + 1 ensures days start at 1, and not at 0
    ygrid = np.arange(int(24 / scaling) + 1)  # hours (sampled or not) + extra 1

    ax.pcolormesh(xgrid, ygrid, value, cmap=colormap, vmin=MIN, vmax=MAX)
    # Invert the vertical axis
    ax.set_ylim(int(24 / scaling), 0)
    # Set tick positions for both axes
    ax.yaxis.set_ticks([])  # [i for i in range(int(24/scaling))]
    ax.xaxis.set_ticks([])
    # Remove ticks by setting their length to 0
    ax.yaxis.set_tick_params(length=0)
    ax.xaxis.set_tick_params(length=0)

    # Remove all spines
    ax.set_frame_on(False)


def plot_heatmap_cfdiff():
    # Here 1 row year/variable and 12 columns for month
    fig, axes = plt.subplots(1, 12, figsize=(14, 5), sharey=True)
    plt.tight_layout()

    for i, year in enumerate([2013]):
        for j, month in enumerate(range(1, 13)):
            # print(f'j: {j}, month: {month}')
            heatmap_diff(df, month, year, axes[j])

    # Adjust margin and space between subplots (extra space is on the left for a label)
    fig.subplots_adjust(
        left=0.05, right=0.98, top=0.9, hspace=0.08, wspace=0
    )  # wspace=0 stacks individual months together but easy to split

    # some room for the legend in the bottom
    fig.subplots_adjust(bottom=0.2)

    # Create a new axis to contain the color bar
    # Values are: (x coordinate of left border, y coordinate for bottom border, width, height)
    cbar_ax = fig.add_axes([0.3, 0.03, 0.4, 0.04])

    # Create a normalizer that goes from minimum to maximum value
    norm = mc.Normalize(MIN, MAX)  # for CFE, otherwise (MIN, MAX)

    # Create the colorbar and set it to horizontal
    cb = fig.colorbar(
        ScalarMappable(norm=norm, cmap=colormap),
        cax=cbar_ax,  # Pass the new axis
        orientation="horizontal",
    )

    # Remove tick marks and set label
    cb.ax.xaxis.set_tick_params(size=0)
    cb.set_label(f"{carrier} hourly capacity factor difference", size=12)

    # add some figure labels and title
    fig.text(0.5, 0.15, "Days of year", ha="center", va="center", fontsize=14)
    fig.text(
        0.03,
        0.5,
        "Hours of a day",
        ha="center",
        va="center",
        rotation="vertical",
        fontsize=14,
    )
    fig.suptitle(
        f"Resource potential diff |  Denmark/Ireland datacenter pair",
        fontsize=20,
        y=0.98,
    )

    # path = snakemake.output.plot.split('capacity.pdf')[0] + f'heatmaps'
    # if not os.path.exists(path):
    #    os.makedirs(path)

    fig.savefig("test.pdf", bbox_inches="tight", transparent=True)

In [ ]:
plot_heatmap_cfdiff()

### Hmm.. let's take a look at correlation

In [ ]:
def retrieve_nb(n, node):
    """
    Retrieve nodal energy balance per hour
        -> lines and links are bidirectional AND their subsets are exclusive.
        -> links include fossil gens
    NB {-1} multiplier is a nodal balance sign
    """

    components = ["Generator", "Load", "StorageUnit", "Store", "Link", "Line"]
    nodal_balance = pd.DataFrame(index=n.snapshots)

    for i in components:
        if i == "Generator":
            node_generators = n.generators.query("bus==@node").index
            nodal_balance = nodal_balance.join(n.generators_t.p[node_generators])
        if i == "Load":
            node_loads = n.loads.query("bus==@node").index
            nodal_balance = nodal_balance.join(-1 * n.loads_t.p_set[node_loads])
        if i == "Link":
            node_export_links = n.links.query("bus0==@node").index
            node_import_links = n.links.query("bus1==@node").index
            nodal_balance = nodal_balance.join(-1 * n.links_t.p0[node_export_links])
            nodal_balance = nodal_balance.join(-1 * n.links_t.p1[node_import_links])
            ##################
        if i == "StorageUnit":
            # node_storage_units = n.storage_units.query('bus==@node').index
            # nodal_balance = nodal_balance.join(n.storage_units_t.p_dispatch[node_storage_units])
            # nodal_balance = nodal_balance.join(n.storage_units_t.p_store[node_storage_units])
            continue
        if i == "Line":
            continue
        if i == "Store":
            continue

    nodal_balance = nodal_balance.rename(columns=rename).groupby(level=0, axis=1).sum()

    # Custom groupby function
    def custom_groupby(column_name):
        if column_name.startswith("vcc"):
            return "spatial shift"
        return column_name

    # Apply custom groupby function
    nodal_balance = nodal_balance.groupby(custom_groupby, axis=1).sum()

    # revert nodal balance sign for display
    if "spatial shift" in nodal_balance.columns:
        nodal_balance["spatial shift"] = nodal_balance["spatial shift"] * -1
    if "temporal shift" in nodal_balance.columns:
        nodal_balance["temporal shift"] = nodal_balance["temporal shift"] * -1

    return nodal_balance

In [ ]:
rename = {}

In [ ]:
datacenters = snakemake.config["ci"]["datacenters"]
locations = list(datacenters.keys())
names = list(datacenters.values())
spatial_shift = retrieve_nb(n, names[1]).get("spatial shift")

In [ ]:
df[f"Denmark {carrier}"][:720].plot()

In [ ]:
spatial_shift[:720].plot()

In [ ]:
# Positive shift -> sending jobs AWAY; negative shift -> receiving jobs
# for Denmark
spatial_shift

In [ ]:
# Positive value -> Denmark is better resources; negative values -> Ireland has better resources
value = df[f"Denmark {carrier}"] - df[f"Ireland {carrier}"]
value.plot()

In [ ]:
# Calculate Pearson correlation coefficient
corr_matrix = np.corrcoef(spatial_shift, -value)
corr_matrix

In [ ]:
from scipy import stats

# Assuming value and spatial_shift are your data arrays
correlation_coefficient, p_value = stats.pearsonr(spatial_shift, -value)

print(f"Correlation Coefficient: {correlation_coefficient}")
print(f"p-value: {p_value}")

In [ ]:
corr_matrix = np.corrcoef(df[f"Denmark {carrier}"], df[f"Ireland {carrier}"])
corr_matrix